In [4]:
import tensorflow as tf
import numpy as np

In [3]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [65]:
# data= open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [5]:
with open('data.txt', 'r', encoding='utf-8') as file:
    data= file.read()
print(data)


The Indian Army is the land-based branch of the Indian Armed Forces.
It is the second-largest standing army in the world, with over 1.3 million active personnel.

The Indian Army's mission is to ensure national security and defend the country's sovereignty.
It plays a crucial role in maintaining peace along the borders and providing internal security during emergencies.

The President of India serves as the Supreme Commander of the Indian Army.
The Chief of Army Staff (COAS) is the highest-ranking officer responsible for the army's operations.

The Indian Army traces its roots back to ancient times, with a long and illustrious history.
It has participated in various wars, including major conflicts like World War I and World War II.

The army has different commands, such as Northern, Southern, Eastern, Western, and Central Command.
Each command is responsible for a specific geographical area and operates under a designated General Officer Commanding-in-Chief (GOC-in-C).

The Indian Arm

In [6]:
vocab = sorted(set(data))
print(f'{len(vocab)} unique characters')

62 unique characters


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(data)

In [8]:
data=data.split('\n')
print(data[:10])

['', 'The Indian Army is the land-based branch of the Indian Armed Forces.', 'It is the second-largest standing army in the world, with over 1.3 million active personnel.', '', "The Indian Army's mission is to ensure national security and defend the country's sovereignty.", 'It plays a crucial role in maintaining peace along the borders and providing internal security during emergencies.', '', 'The President of India serves as the Supreme Commander of the Indian Army.', "The Chief of Army Staff (COAS) is the highest-ranking officer responsible for the army's operations.", '']


In [9]:
input_sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [10]:
max_sequence_len = max([len(x) for x in input_sequences])

In [95]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequences =np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [96]:
input_sequences.shape

(19490, 148)

In [97]:
xs= input_sequences[:,:-1]
labels = input_sequences[:,-1]

In [98]:
total_chars= len(tokenizer.word_index)+1

In [99]:
ys = tf.keras.utils.to_categorical(labels, num_classes=total_chars)

In [100]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(total_chars,256,input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_chars,activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

2023-07-26 14:52:23.304685: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-26 14:52:23.313898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-26 14:52:23.321217: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [110]:
history = model.fit(xs,ys,epochs=5)

Epoch 1/5
610/610 [==============================] - 101s 165ms/step - loss: 0.2185 - accuracy: 0.9387
Epoch 2/5
610/610 [==============================] - 97s 158ms/step - loss: 0.2101 - accuracy: 0.9403
Epoch 3/5
610/610 [==============================] - 99s 162ms/step - loss: 0.1993 - accuracy: 0.9445
Epoch 4/5
610/610 [==============================] - 100s 163ms/step - loss: 0.1838 - accuracy: 0.9470
Epoch 5/5
610/610 [==============================] - 96s 157ms/step - loss: 0.1918 - accuracy: 0.9448


In [115]:
seed_text=input('Enter the seed text: ')
next_words=int(input('Enter the number of chars to be generated: '))
for _ in range(next_words):
    token_list=tokenizer.texts_to_sequences([seed_text])[0]
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    output_word=tokenizer.index_word[np.argmax(predicted, axis=-1)[0]]
    seed_text+=output_word
print(seed_text)    

engineering cont its personnel to cadetes and their families 


In [113]:
model.save('tg.h5')

In [2]:
import pickle

In [11]:
with open('tokenizer_data.pkl', 'wb') as handle:
    pickle.dump(
        {'tokenizer': tokenizer, 'num_words':vocab ,'max_len':max_sequence_len}, handle)